In [1]:
%%html
<style>
.jp-LabShell {
    display: none !important;
}
</style>

In [2]:
import ipywidgets as widgets
from IPython.display import display
import plotly.express as px
import pandas as pd
import plotly.io as pio

# Настройки для Voilà
pio.renderers.default = 'notebook'


In [3]:

# Загрузка данных (оставляем parquet)
df = pd.read_parquet('data/data.parquet')  # Убедитесь, что путь правильный

# Виджеты
dropdown1 = widgets.Dropdown(
    options=['Все'] + sorted(df['Месяц'].unique().tolist()),
    description='Месяц:'
)

dropdown2 = widgets.Dropdown(
    options=['Все'] + sorted(df['Направление'].unique().tolist()),
    description='Направление:'
)

dropdown3 = widgets.Dropdown(
    options=['Все'],
    description='Станция:',
    disabled=True
)

output = widgets.Output()

def update_stations(change):
    if dropdown2.value != 'Все':
        stations = df[df['Направление'] == dropdown2.value]['Станция'].unique()
        dropdown3.options = ['Все'] + sorted(stations.tolist())
        dropdown3.disabled = False
    else:
        dropdown3.disabled = True
    update_plot()

def update_plot():
    filtered = df.copy()
    if dropdown1.value != 'Все':
        filtered = filtered[filtered['Месяц'] == dropdown1.value]
    if dropdown2.value != 'Все':
        filtered = filtered[filtered['Направление'] == dropdown2.value]
    if not dropdown3.disabled and dropdown3.value != 'Все':
        filtered = filtered[filtered['Станция'] == dropdown3.value]
    
    with output:
        output.clear_output()
        fig = px.bar(
            filtered,
            x='Месяц',
            y='Количество',
            title=f"Отмены: {dropdown2.value or 'Все направления'}"
        )
        fig.show()

dropdown1.observe(lambda _: update_plot(), 'value')
dropdown2.observe(lambda _: update_stations(_), 'value')
dropdown3.observe(lambda _: update_plot(), 'value')

# Инициализация
display(widgets.VBox([dropdown1, dropdown2, dropdown3]))
display(output)
update_plot()  # Первый запуск

Output()